In [4]:
from typing import Annotated , TypedDict , Optional
from langgraph.graph import add_messages , StateGraph , END
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4 #Its similar like manam thread id istam ga ala
import json

load_dotenv()

True

In [22]:
from langchain_core.messages import AIMessage , HumanMessage , ToolMessage
from langgraph.prebuilt import ToolNode , tools_condition

model = ChatGroq(model = "llama-3.1-8b-instant")
search_tool = TavilySearchResults(max_results=4)

tool_search= [search_tool]

memory = MemorySaver()
tools = ToolNode(tools=tool_search)

llm_with_tools = model.bind_tools(tools=tool_search)


class State(TypedDict):
    messages: Annotated[list , add_messages]

def call_model(state: State):
    return {"messages": llm_with_tools.invoke(state["messages"])}


graph_builder = StateGraph(State)
graph_builder.add_node("model" , call_model)
graph_builder.add_node("tools" , tools)
graph_builder.set_entry_point("model")

# graph_builder.add_conditional_edges("model" , ) #I think this is not needed in newer version

graph_builder.add_conditional_edges("model" , tools_condition)

graph_builder.add_edge("tools" , "model")

graph = graph_builder.compile(checkpointer=memory)

In [29]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	model(model)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> model;
	model -.-> __end__;
	model -.-> tools;
	tools --> model;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [26]:
config = {
    "configurable":{
        "thread_id":7
    }
}

async for event in graph.astream_events({
    "messages":[HumanMessage(content="When is the spacex launch")]
} , config=config):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='When is the spacex launch', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '4283758a-06d1-48c5-b7c2-fa485b405cee', 'metadata': {'thread_id': 7}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='When is the spacex launch', additional_kwargs={}, response_metadata={}, id='b0e31626-e372-4355-b41d-7eda5af15307')]}}, 'name': 'model', 'tags': ['graph:step:1'], 'run_id': '4086e9ac-efff-44b2-b49c-5d0849e2dc8c', 'metadata': {'thread_id': 7, 'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:e5e68e4a-37b0-6ba6-7b1a-a9c0f1b5c92d'}, 'parent_ids': ['4283758a-06d1-48c5-b7c2-fa485b405cee']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='When is the spacex launch', addi

In [28]:
graph.invoke({"messages": "When is spacex launched"} , config = config)

{'messages': [HumanMessage(content='When is the spacex launch', additional_kwargs={}, response_metadata={}, id='b0e31626-e372-4355-b41d-7eda5af15307'),
  AIMessage(content='I don\'t have access to real-time information. However, I can suggest an alternative. <function=tavily_search_results_json>{"query": "spacex launch date"}</function>', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a7a2f9abbf', 'service_tier': 'on_demand'}, id='run--a91b15f7-ff7b-4ced-b9bb-ec8be548fa53', usage_metadata={'input_tokens': 284, 'output_tokens': 40, 'total_tokens': 324}),
  HumanMessage(content='When is spacex launched', additional_kwargs={}, response_metadata={}, id='09e985fd-78c0-45c1-86dc-43766a89498c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '3x1nd4x0n', 'function': {'arguments': '{"query":"spacex latest launch date"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_met